In [ ]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.8 MB/s eta 0:00:00


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report
import numpy as np
import nltk
nltk.download('brown')
nltk.download('punkt')  # Optional, if tokenizing
nltk.download('universal_tagset')  # For Universal POS tagging

from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from collections import defaultdict
#import numpy as np
#import pandas as pd
from sklearn.metrics import confusion_matrix
#mport seaborn as sns
#mport matplotlib.pyplot as plt
import gradio as gr
from sklearn.model_selection import KFold

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [ ]:
class POSTagger:

    def __init__(self, train_data, test_data, tagset) -> None:

        self.test_data = test_data
        self.train_data = train_data
        self.tagset = tagset
        self.tag_count = self.getTagCount()
        self.lexical_prob = self.lexical_probability()
        self.transition_prob = self.transition_probability()
        self.start_prob = self.start_probability()
        self.end_prob = self.end_probability()

    def getTagCount(self):
        tag_count = defaultdict(int)
        for sent in self.train_data:
            for (w, t) in sent:
                tag_count[t] += 1
        return tag_count

    def lexical_probability(self):
        lexical_prob = defaultdict(lambda: defaultdict(float))
        for s in self.train_data:
            for (w, t) in s:
                lexical_prob[w.lower()][t] += 1
        return lexical_prob

    def transition_probability(self):
        transition_prob = defaultdict(lambda: defaultdict(float))
        for sent in self.train_data:
            for (w1, t1), (w2, t2) in nltk.bigrams(sent):
                transition_prob[t2][t1] += 1
        for t2 in self.tagset:
            for t1 in self.tagset:
                transition_prob[t2][t1] /= self.tag_count[t1]
        return transition_prob

    def start_probability(self):
        start_prob = defaultdict(float)
        for sent in self.train_data:
            start_prob[sent[0][1]] += 1
        for t in self.tagset:
            start_prob[t] /= len(self.train_data)
        return start_prob

    def end_probability(self):
        end_prob = defaultdict(float)
        for sent in self.train_data:
            end_prob[sent[len(sent)-1][1]] += 1
        for t in self.tagset:
            end_prob[t] /= len(self.train_data)
        return end_prob

    def viterbi(self, words):
        dp = [defaultdict(float) for _ in range(len(words) + 1)]
        backpointers = [defaultdict(int) for _ in range(len(words)+1)]

        prod = 0
        for t in self.tagset:
            prod += self.lexical_prob[words[0]][t]
        if prod == 0:
            for t in self.tagset:
                dp[0][t] = self.start_prob[t]
        else:
            for t in self.tagset:
                dp[0][t] = self.start_prob[t] * self.lexical_prob[words[0]][t]/self.tag_count[t]

        for i in range(1, len(words)):
            prod = 0
            for t in self.tagset:
                prod += self.lexical_prob[words[i]][t]
            if prod == 0:
                for t in self.tagset:
                    dp[i][t], backpointers[i][t] = max((dp[i-1][prev_t] * self.transition_prob[t][prev_t] , prev_t) for prev_t in self.tagset)
                continue
            for t in self.tagset:
                dp[i][t], backpointers[i][t] = max((dp[i-1][prev_t] * self.transition_prob[t][prev_t] * self.lexical_prob[words[i]][t]/self.tag_count[t], prev_t) for prev_t in self.tagset)

        dp[len(words)]['.'], backpointers[len(words)]['.'] = max((dp[len(words)-1][prev_t] * self.end_prob[prev_t], prev_t) for prev_t in self.tagset)
        best_path = [backpointers[len(words)]['.']]

        for i in range(len(words)-1, 0, -1):
            best_path.append(backpointers[i][best_path[-1]])

        best_path.reverse()

        return best_path
    def evaluate(self):
      true_tags = []
      predicted_tags = []

      for sentence in self.test_data:
        words = [w for w, t in sentence]
        true_tags_sentence = [t for w, t in sentence]
        predicted_tags_sentence = self.viterbi(words)

        true_tags.extend(true_tags_sentence)
        predicted_tags.extend(predicted_tags_sentence)

    # Calculate overall metrics
      overall_precision = precision_score(true_tags, predicted_tags, average='weighted', zero_division=0)
      overall_recall = recall_score(true_tags, predicted_tags, average='weighted', zero_division=0)
      overall_f1 = f1_score(true_tags, predicted_tags, average='weighted', zero_division=0)

    # Confusion matrix
      conf_matrix = confusion_matrix(true_tags, predicted_tags, labels=self.tagset)

    # Print overall metrics
      print("Overall Precision:", overall_precision)
      print("Overall Recall:", overall_recall)
      print("Overall F1 Score:", overall_f1)
      print("Confusion Matrix:\n", conf_matrix)

    # Calculate metrics for individual tags
      print("\nMetrics for Individual Tags:")
      report = classification_report(true_tags, predicted_tags, labels=self.tagset)
      print(report)

    # Optionally you can add your test method here (omitted for brevity)



In [ ]:
if __name__ == "__main__":
    # Load data and initialize tagger
    data = brown.tagged_sents(tagset='universal')
    tagset = ['PRT', 'ADP', 'DET', 'ADV', 'PRON', '.', 'NUM', 'NOUN', 'ADJ', 'X', 'VERB', 'CONJ']
    tagger = POSTagger(data, data, tagset)
    tagger.evaluate()
    # Function to predict POS tags for Gradio
    def predict(sentence):
        words = [word.lower() for word in word_tokenize(sentence)]
        pos_tags = tagger.viterbi(words)
        return list(zip(words, pos_tags))  # Pair words with their POS tags

    # Gradio Interface
    interface = gr.Interface(
        fn=predict,
        inputs="text",
        outputs="text",
        title="POS Tagger",
        description="Input a sentence to get POS tags."
    )
    interface.launch()

Overall Precision: 0.9223084474368672
Overall Recall: 0.9194594864587424
Overall F1 Score: 0.9195131566826688
Confusion Matrix:
 [[ 24153   2349    354    261   1692     24      0    682     43     15
     256      0]
 [  2194 136754   2274   1823    892     72      1    368     72     64
     112    140]
 [     1   1427 132235    224   2657      9      2    357     15     48
      30     14]
 [   633   3152    953  45893    704     43      0   2537   2105     15
     124     80]
 [   197    442    495    366  46142     12      0    813      7      6
     854      0]
 [     0      0      0      0      0 147482      0      0      0     83
       0      0]
 [     0     37    406      8     49     11  13848    493     12      6
       4      0]
 [   278   4761  13094    288   6799   1055    305 238229   5282    163
    5303      1]
 [   180    581   3772   2120    136     38      4   3055  73305     35
     495      0]
 [     3     66    113      2      9     21      0    234     38    86

**this code handles unknown words handling(changes  made in viterbi function) and upper case already done(in predict function) and K fold cross validation**

In [ ]:
class POSTagger:

    def __init__(self, train_data, test_data, tagset) -> None:

        self.test_data = test_data
        self.train_data = train_data
        self.tagset = tagset
        self.tag_count = self.getTagCount()
        self.lexical_prob = self.lexical_probability()
        self.transition_prob = self.transition_probability()
        self.start_prob = self.start_probability()
        self.end_prob = self.end_probability()

    def getTagCount(self):
        tag_count = defaultdict(int)
        for sent in self.train_data:
            for (w, t) in sent:
                tag_count[t] += 1
        return tag_count

    def lexical_probability(self):
        lexical_prob = defaultdict(lambda: defaultdict(float))
        for s in self.train_data:
            for (w, t) in s:
                lexical_prob[w.lower()][t] += 1
        return lexical_prob

    def transition_probability(self):
        transition_prob = defaultdict(lambda: defaultdict(float))
        for sent in self.train_data:
            for (w1, t1), (w2, t2) in nltk.bigrams(sent):
                transition_prob[t2][t1] += 1
        for t2 in self.tagset:
            for t1 in self.tagset:
                transition_prob[t2][t1] /= self.tag_count[t1]
        return transition_prob

    def start_probability(self):
        start_prob = defaultdict(float)
        for sent in self.train_data:
            start_prob[sent[0][1]] += 1
        for t in self.tagset:
            start_prob[t] /= len(self.train_data)
        return start_prob

    def end_probability(self):
        end_prob = defaultdict(float)
        for sent in self.train_data:
            end_prob[sent[len(sent)-1][1]] += 1
        for t in self.tagset:
            end_prob[t] /= len(self.train_data)
        return end_prob

    def viterbi(self, words):
        dp = [defaultdict(float) for _ in range(len(words) + 1)]
        backpointers = [defaultdict(int) for _ in range(len(words)+1)]

        # Initialize for the first word
        for t in self.tagset:
            dp[0][t] = self.start_prob[t] * self.lexical_prob[words[0]][t] / self.tag_count[t] if words[0] in self.lexical_prob else self.start_prob[t] * 0.1  # Default probability for unknown words

        for i in range(1, len(words)):
            for t in self.tagset:
                if words[i] in self.lexical_prob:
                    dp[i][t], backpointers[i][t] = max((dp[i-1][prev_t] * self.transition_prob[t][prev_t] * self.lexical_prob[words[i]][t] / self.tag_count[t], prev_t) for prev_t in self.tagset)
                else:
                    # Handling unknown words
                    dp[i][t], backpointers[i][t] = max((dp[i-1][prev_t] * self.transition_prob[t][prev_t] * 0.1, prev_t) for prev_t in self.tagset)  # Default probability for unknown words

        dp[len(words)]['.'], backpointers[len(words)]['.'] = max((dp[len(words)-1][prev_t] * self.end_prob[prev_t], prev_t) for prev_t in self.tagset)
        best_path = [backpointers[len(words)]['.']]

        for i in range(len(words)-1, 0, -1):
            best_path.append(backpointers[i][best_path[-1]])

        best_path.reverse()

        return best_path

    def evaluate(self):
      true_tags = []
      predicted_tags = []

      for sentence in self.test_data:
        words = [w for w, t in sentence]
        true_tags_sentence = [t for w, t in sentence]
        predicted_tags_sentence = self.viterbi(words)

        true_tags.extend(true_tags_sentence)
        predicted_tags.extend(predicted_tags_sentence)

    # Calculate overall metrics
      overall_precision = precision_score(true_tags, predicted_tags, average='weighted', zero_division=0)
      overall_recall = recall_score(true_tags, predicted_tags, average='weighted', zero_division=0)
      overall_f1 = f1_score(true_tags, predicted_tags, average='weighted', zero_division=0)

    # Confusion matrix
      conf_matrix = confusion_matrix(true_tags, predicted_tags, labels=self.tagset)

    # Print overall metrics
      print("Overall Precision:", overall_precision)
      print("Overall Recall:", overall_recall)
      print("Overall F1 Score:", overall_f1)
      print("Confusion Matrix:\n", conf_matrix)

    # Calculate metrics for individual tags
      print("\nMetrics for Individual Tags:")
      report = classification_report(true_tags, predicted_tags, labels=self.tagset)
      print(report)

    # Optionally you can add your test method here (omitted for brevity)


here we includes K-fold cross validation

In [ ]:
if __name__ == "__main__":
    # Load data
    data = brown.tagged_sents(tagset='universal')
    tagset = ['PRT', 'ADP', 'DET', 'ADV', 'PRON', '.', 'NUM', 'NOUN', 'ADJ', 'X', 'VERB', 'CONJ']

    # K-Fold Cross-Validation
    kf = KFold(n_splits=5)
    for train_index, test_index in kf.split(data):
        train_data = [data[i] for i in train_index]
        test_data = [data[i] for i in test_index]

        # Initialize and evaluate POSTagger
        tagger = POSTagger(train_data, test_data, tagset)
        tagger.evaluate()

    # Function to predict POS tags for Gradio
    def predict(sentence):
        words = [word.lower() for word in nltk.word_tokenize(sentence)]
        pos_tags = tagger.viterbi(words)
        return list(zip(words, pos_tags))  # Pair words with their POS tags

    # Gradio Interface
    interface = gr.Interface(
        fn=predict,
        inputs="text",
        outputs="text",
        title="POS Tagger",
        description="Input a sentence to get POS tags."
    )
    interface.launch()

Overall Precision: 0.8937750864531743
Overall Recall: 0.8872184054960857
Overall F1 Score: 0.8869386333483691
Confusion Matrix:
 [[ 4765   558    52    46   252     3     0    69    17     1    28     0]
 [  432 29459   445   347   217     6     0    72    13     4    21    32]
 [    1   448 28182    47   668     1     0    91     2     4     8     3]
 [   87   596   148  8964   174    13     0   489   449     1    40    10]
 [    0   131    98    66  7662     2     0    99     0     0   146     0]
 [    0     0     0     0     0 30110     0     0     0     5     0     0]
 [    0    39   200     2    27     7  3309   322    48     1    19     0]
 [    6  2227  4950    68  1681   489    48 52995  2181    23  2122     0]
 [   28   298  1411   489    69    46     4  1231 15184     5   176     0]
 [    0    23    48     0     2     3     0   106    13    81     5     0]
 [    1   431   309    54    60    61     0  1579   257     6 34370     0]
 [    0   239    75    74    94     2     0   